<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/philipp/notebooks/Dashboard_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q&A Dasbboard / Live Demo

In [4]:
%%capture
!pip install gradio
# !pip install git+https://github.com/openai/whisper.git
# import whisper
import gradio as gr
import time
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
import webbrowser

# Gemini API Setup
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [5]:
def reformulate_question(text):
    prompt = f"""Please carefully take the following statement and reformulate
        it into a clear, concise, and grammatically correct question that
        directly addresses the user, while maintaining the same meaning as the
        original statement. If the provided text is already a question, retain
        its original meaning without changing it. Do not omit or alter any
        content or facts. The statement is: '{text}'"""

    try:
        response = model.generate_content(prompt)
        time.sleep(4)
        return response.text.strip()
    except Exception as e:
        exit("Error during API call: ", e)

In [6]:
dfs = list()

for q in range(1, 6):
    url = f'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/questionnaires/questionnaire{q}.json'
    df = pd.read_json(url)
    df['questionnaire'] = f"Questionnaire {q}"
    df = df[['questionnaire', 'question']]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df['better_question'] = df['question'].apply(reformulate_question)

In [7]:
answered_questions = list()


def chatbot(selected_option, input_message, chat_history):
    if not chat_history and input_message != "start":
        response = "Select a questionnaire and start the conversation with 'start'."
        chat_history.append((input_message, response))
        return chat_history, chat_history

    if chat_history and not input_message:
        response = "No worries! It seems like you’re done with the questions. 😊 The chat will end now. Feel free to return anytime!"
        chat_history.append((input_message, response))
        return chat_history, chat_history

    all_questions = df[df['questionnaire'] == selected_option]['better_question'].tolist()

    if input_message in all_questions: # Todo: Das natürlich noch Quatsch
        answered_questions.append(input_message)
    print("reached")
    unanswered_questions = [q for q in all_questions if q not in answered_questions]

    if len(unanswered_questions) == 0:
        response = "You have answered all the questions in this questionnaire."
        chat_history.append((input_message, response))
        return chat_history, chat_history
    elif len(unanswered_questions) < 3:
        response = "If you don't want to answer the other questions, end the conversation with 'end'.\n"
    else:
        response = ""

    unanswered_questions_string = '\n'.join(unanswered_questions)
    response = response + "Please answer the following questions:\n" + unanswered_questions_string

    chat_history.append((input_message, response))
    return chat_history, chat_history


with gr.Blocks() as app:
    gr.Markdown("# 🎯 **SurveyMaster 8** - Your Ultimate Polling Chatbot 🤖")

    option = gr.Dropdown(
        list(df['questionnaire'].unique()),
        label="Choose a questionnaire"
    )

    chatbot_interface = gr.Chatbot()

    message = gr.Textbox(label="Your answers", placeholder="Answer the questions ...")

    send_button = gr.Button("Send")

    chat_history = gr.State([])

    send_button.click(
        fn=chatbot,
        inputs=[option, message, chat_history],
        outputs=[chatbot_interface, chat_history]
    )

app.launch()
webbrowser.open(app.share_url)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed2cd06cd4665db434.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


False